In [ ]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import time

%matplotlib inline

### Useful functions ###

In [ ]:
#convert shot flags to colors
def flag_colors(flag):
    if flag == 1: return 'green'
    if flag == 0: return 'red'
    return 'black'

In [333]:
# convert matchup string to Home=1, away=0
# example: LAL @ POR: home=0
# example: LAL vs. POR: home=1

def get_home_away(string):
    if '@' in string: return 0 #away
    elif 'vs' in string : return 1 #home
    else: return "ACORDA CRL"


In [475]:
# convert season in format '1999-00' to 
# example: '1999-00': 3
def get_season_num(year):
    # year is in format '2009-10'
    year0 = 1996 #hardcodado
    y = year[:4]
    y_int = int(y)
    
    return y_int - year0
    
    

In [477]:
#quick asserts
assert get_season_num('2010-11') == 14
assert get_home_away('LAL @ POR') == 0
assert get_home_away('LAL vs UTA') == 1

# Import data

In [ ]:
data = pd.read_csv('data.csv')

In [ ]:
data.head()

In [ ]:
list(data.columns.values)

# Notes

* Can we improve the missing data strategy?
* Are all the limitations to algorithm application verified?

* prevent leakage: train only on shots prior to the ones we're predicting (year)
* select features and then explore number of neighbours

* target: score=0.75

# Preprocessing

In [ ]:
# TRAINING AND TEST DATA
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 0)

In [ ]:
# MISSING DATA
# don't consider rows with shot_made_flag = NaN
train_data = train_data[np.isfinite(train_data['shot_made_flag'])]
test_data = test_data[np.isfinite(test_data['shot_made_flag'])]

   # Exploring features

### Opponent

In [ ]:
total_shots = pd.pivot_table(data, index="opponent", values="shot_id", aggfunc='count')

In [ ]:
shots_scored = pd.pivot_table(data, index="opponent", values="shot_made_flag", aggfunc='sum')

In [ ]:
accuracy = shots_scored/total_shots

In [ ]:
accuracy.mean()

In [ ]:
total_shots.plot.bar(title='Kobe Bryant\ntotal shots by opponent')

In [ ]:
shots_scored.plot.bar(title='Kobe Bryant\nshots scored by opponent')

In [ ]:
accuracy.plot.bar(title='Kobe Bryant\n% by opponent')

#### Comments:

* The opponent doesn't seem to influence the accuracy

* also, spikes for long-time shitty teams: NYK, CLE, GSW(!), PHX, SAC

### Shots by distance

In [ ]:
total_distance = pd.pivot_table(data, index="shot_distance", values="shot_id", aggfunc='count')

In [ ]:
shots_scored = pd.pivot_table(data, index="shot_distance", values="shot_made_flag", aggfunc='sum')

In [ ]:
accuracy = shots_scored/total_distance

In [ ]:
total_distance.plot.bar(title='Kobe Bryant\ntotal shots by distance')
plt.axis([0, 30, 0, 6000])

In [ ]:
shots_scored.plot.bar(title='Kobe Bryant\nshots scored by distance')
plt.axis([0, 30, 0, 3500])

In [ ]:
accuracy.plot.bar(title='Kobe Bryant\n% by distance')
plt.axis([0, 30, 0, 1])

#### Comments:

* ~~Only 30 appears to have a very different percentage~~ now looking sweeter, but still not good - certainly didn't miss half his dunks for his career!
* PM: totally agree!

### Minutes remaining

In [ ]:
minutes = pd.pivot_table(data, index="minutes_remaining", values="shot_id", aggfunc='count')

In [ ]:
shots_scored = pd.pivot_table(data, index="minutes_remaining", values="shot_made_flag", aggfunc='sum')

In [ ]:
accuracy = shots_scored/minutes

In [ ]:
minutes.plot.bar(title='Kobe Bryant\nAttempts byminutes remaining')

In [ ]:
shots_scored.plot.bar(title='Kobe Bryant\nshots scored by minutes remaining')

In [ ]:
accuracy.plot.bar(title='Kobe Bryant\n% by minutes remaining')
plt.axis([0, 11, 0, .5])

### Seconds remaining

In [405]:
seconds = pd.pivot_table(data, index=[ "seconds_remaining", 'period'], values="shot_id", aggfunc='count')

In [406]:
shots_scored = pd.pivot_table(data, index=[ "seconds_remaining", 'period'], values="shot_made_flag", aggfunc='sum')

In [408]:
accuracy = shots_scored/seconds
accuracy

seconds_remaining  period
0                  1         0.275093
                   2         0.261194
                   3         0.275591
                   4         0.263736
                   5         0.500000
                   6         0.500000
1                  1         0.342246
                   2         0.328859
                   3         0.375000
                   4         0.296000
                   5         0.076923
                   6         1.000000
2                  1         0.372263
                   2         0.358108
                   3         0.320988
                   4         0.256944
                   5         0.000000
                   7         0.000000
3                  1         0.381679
                   2         0.322314
                   3         0.397163
                   4         0.304348
                   5         0.272727
                   6         0.000000
4                  1         0.444444
                   2    

### Shots by zone range

In [ ]:
shot_zones = pd.pivot_table(data, index="shot_zone_range", values="shot_id", aggfunc='count')
shot_zones_pct = shot_zones/shot_zones.sum()

# change order
shot_zones_pct = shot_zones_pct.reindex(index=['Less Than 8 ft.',  u'8-16 ft.', u'16-24 ft.', u'24+ ft.', u'Back Court Shot'])
shot_zones_pct.plot.bar(title='Kobe Bryant\n% by zone range')

In [ ]:
# let's try plotting all of his shots and show if miss or hit

shots_pos = data[['loc_x', 'loc_y']][data.shot_made_flag==1]
shot_colors = map(flag_colors, data.shot_made_flag)
shots_pos.plot.scatter(x='loc_x', y='loc_y', s=10, c='g', ylim=(0,400), figsize=(20,10))


#### Comments:

* Front, 45 degrees or 90 degrees
* Heat map to understand accuracy

### Shots by action type

In [ ]:
shots_actionType = pd.pivot_table(data, index="action_type", values="shot_id", aggfunc='count')
shots_actionType.plot.bar(title='Kobe Bryant\naction type')

### Combined shot type

In [ ]:
shots_combined = pd.pivot_table(data, index="combined_shot_type", values="shot_id", aggfunc='count')
shots_combined.plot.bar(title='Kobe Bryant\ncombined shot type')

In [ ]:
combined_scored = pd.pivot_table(data, index="combined_shot_type", values="shot_made_flag", aggfunc='sum')
combined_scored.plot.bar(title='Kobe Bryant\nscored combined shot type')

In [ ]:
accuracy = combined_scored / shots_combined
accuracy.plot.bar(title='Kobe Bryant\n% combined shot type')

### Shots by period

In [ ]:
shots_period = pd.pivot_table(data, index="period", values="shot_id", aggfunc='count')
shots_period.plot.bar(title='Kobe Bryant\nshots by period')

In [ ]:
period_scored = pd.pivot_table(data, index="period", values="shot_made_flag", aggfunc='sum')
period_scored.plot.bar(title='Kobe Bryant\nscored by period')

In [ ]:
accuracy = period_scored/shots_period
accuracy.plot.bar(title='Kobe Bryant\n% by period')

### Playoffs

In [ ]:
accuracy = playoffs_scored/shots_playoffs
accuracy.plot.bar(title='Kobe Bryant\n% by playoff')
accuracy

### Shot type

In [ ]:
shot_type = pd.pivot_table(data, index="shot_type", values="shot_id", aggfunc='count')
shot_type.plot.bar(title='Kobe Bryant\nshot type')

In [ ]:
shotType_scored = pd.pivot_table(data, index="shot_type", values="shot_made_flag", aggfunc='sum')
shotType_scored.plot.bar(title='Kobe Bryant\nscored by shot type')

In [ ]:
accuracy = shotType_scored/shot_type
accuracy.plot.bar(title='Kobe Bryant\n% by shot type')

In [ ]:
#season = pd.pivot_table(data, index="season", values="shot_id", aggfunc='count')
#ax1 = season.plot.bar(title='Kobe Bryant\nSeason')

#season_scored = pd.pivot_table(data, index="season", values="shot_made_flag", aggfunc='sum')
#ax2 = season_scored.plot.bar(title='Kobe Bryant\nscored by shot type')

season_accuracy = season_scored/season
ax3 = season_accuracy.plot.bar(title='Kobe Bryant\n% by shot type', ylim=(.30,.45))

# acrescentear subplots

## On temporal series (in construction)

In [ ]:
data['game_date'][0]

In [ ]:
time.strptime("00-12-30", "%y-%m-%d")

## Dunks issue

* Kobe scored 780 dunk from 00/01 to 15/16
* http://www.basketball-reference.com/players/b/bryanko01.html
* Kobe's carrer started in 96/97

In [ ]:
shots_combined = pd.pivot_table(data, index="combined_shot_type", values="shot_id", aggfunc='count')

In [ ]:
shots_combined

In [ ]:
combined_scored = pd.pivot_table(data, index="combined_shot_type", values="shot_made_flag", aggfunc='sum')

In [ ]:
combined_scored

In [ ]:
accuracy = combined_scored/shots_combined

In [ ]:
accuracy

In [ ]:
accuracy.plot.bar()

#### Conclusion:
* We have more successful dunks than basketball reference, which is possible because we have data from 96/97 to 00/01
* In 10 dunks, Kobe misses 2...

### Missing Analysis

* Home/away
* Time of the year
* Location

# 1. kNN

** References: **
* http://scikit-learn.org/stable/modules/neighbors.html
* http://www.amazon.com/Building-Machine-Learning-Systems-Python/dp/1784392774/ref=sr_1_1?ie=UTF8&qid=1461733493&sr=8-1&keywords=luis+pedro+coelho+python

In [ ]:
from sklearn import neighbors

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=2)

In [ ]:
x1 = train_data['loc_x'].reshape(len(train_data['loc_x']),1)
x2 = train_data['loc_y'].reshape(len(train_data['loc_y']),1)
x3 = train_data['shot_distance'].reshape(len(train_data['shot_distance']),1)

In [ ]:
x = np.concatenate((x1,x2,x3), axis=1)
y = train_data['shot_made_flag'].reshape(len(train_data['shot_made_flag']),1)

print x.shape
print y.shape

In [ ]:
knn.fit(x,y.ravel())

In [ ]:
#knn.predict([0])

In [ ]:
#knn.predict_proba([0])

## 1.1 kNN evaluation

In [ ]:
x1_test = test_data['loc_x'].reshape(len(test_data['loc_x']),1)
x2_test = test_data['loc_y'].reshape(len(test_data['loc_y']),1)
x3_test = test_data['shot_distance'].reshape(len(test_data['shot_distance']),1)

In [ ]:
x_test = np.concatenate((x1_test,x2_test,x3_test), axis=1)
y_test = test_data['shot_made_flag'].reshape(len(test_data['shot_made_flag']),1)

In [ ]:
knn.score(x_test,y_test)

In [ ]:
# try different numbers of neighbours
l = []
for i in xrange(1,100):
    knn = neighbors.KNeighborsClassifier(n_neighbors=i)
    knn.fit(x,y.ravel())
    
    sc = knn.score(x_test,y_test)
    l.append(sc)
    
plt.plot(l)

## Knn for distance, shot_type

In [ ]:
train_data.replace(to_replace='2PT Field Goal', value=2, inplace=True)
train_data.replace(to_replace='3PT Field Goal', value=3, inplace=True)

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=3)
x1 = train_data['shot_distance'].reshape(len(train_data['shot_distance']),1)
x2 = train_data['shot_type'].reshape(len(train_data['shot_type']),1)

x = np.concatenate((x1,x2), axis=1)
y = train_data['shot_made_flag'].reshape(len(train_data['shot_made_flag']),1)

knn.fit(x,y.ravel())

In [ ]:
x1_test = train_data['shot_distance'].reshape(len(train_data['shot_distance']),1)
x2_test = train_data['shot_type'].reshape(len(train_data['shot_type']),1)

x_test = np.concatenate((x1_test,x2_test), axis=1)
y_test = train_data['shot_made_flag'].reshape(len(train_data['shot_made_flag']),1)

In [ ]:
knn.score(x_test, y_test)

In [ ]:
knn.fit(x1, y.ravel())
knn.score(x1_test, y_test)

In [ ]:
# try different numbers of neighbours
l = []
for i in xrange(1,100):
    knn = neighbors.KNeighborsClassifier(n_neighbors=i)
    knn.fit(x,y.ravel())
    
    sc = knn.score(x_test,y_test)
    l.append(sc)
    
plt.plot(l)

## PLaying with correlation

In [ ]:
data[['loc_x', 'loc_y']];

In [ ]:
data['angle'] = np.degrees(np.arctan(data['loc_x']/data['loc_y']))
RO = data.corr()

In [ ]:
RO.loc['shot_made_flag']

In [ ]:
data['shot_id' ]

## New feature matrix

In [500]:
# create design matrix
dm = data[['loc_x', 'loc_y', 'shot_distance', 'period', 'season', 'minutes_remaining', 'seconds_remaining', 'game_date', 'matchup']]

In [501]:
# modify design matrix

# convert shot_distance from feet to meters
dm.loc[:,'shot_distance'] = dm['shot_distance'].apply(lambda x: x*0.3048)

# add angle feature and clean NaN by assuming angle=0 when distance=0
dm.loc[:,'angle'] = pd.Series(np.degrees(np.arctan(dm['loc_x']/dm['loc_y'])))
dm['angle'].fillna(0, inplace=True)


# convert matchup to Home/Away
# Home=0, Away=1
dm.loc[:, 'Home'] = dm['matchup'].apply(get_home_away)

# convert seasons to first, second etc
# needs: convert to date 
dm.loc[:,'season'] = dm['season'].apply(get_season_num)

# convert minutes + seconds remaining to time remaining in quarter (in seconds)
dm.loc[:, 'time_remaining'] = dm['minutes_remaining']*60 + dm['seconds_remaining']

# clean dataframe
cols_to_delete = ['loc_x', 'loc_y', 'minutes_remaining', 'seconds_remaining', 'matchup']
dm.drop(cols_to_delete, axis=1, inplace=True)

# temporary: also drop game_date
dm.drop('game_date', axis=1, inplace=True)

/Users/joaopcoelho/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/joaopcoelho/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [502]:
dm

,shot_distance,period,season,angle,Home,time_remaining
0,5.4864,1,4,66.677303,0,627
1,4.5720,1,4,-90.000000,0,622
2,4.8768,1,4,-36.801931,0,465
3,6.7056,1,4,38.258303,0,412
4,0.0000,2,4,0.000000,0,379
5,4.2672,3,4,85.661733,0,572
6,0.0000,3,4,0.000000,0,532
7,0.6096,3,4,2.045408,0,485
8,3.6576,3,4,-31.041710,0,372
9,3.6576,3,4,-14.788678,0,216
